In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
x=np.array(pd.read_csv('Xsvm.csv'))
y=np.array(pd.read_csv('ysvm.csv'))

N=len(y)
x_test=x[N:,:]
y_test=y[N:,:]
x=x[0:N,:]
y=y[0:N,:]

x_Q=np.matrix('1.9, 0.4;0.9, 0.9;1.4, 1.5;0.01, 0.005')

def fit(x,y,N,k_type,sigma=2,c=1,p=4):
    global w,b,sv_al,sv_y,sv_x,kr_type  
    
    kr_type=k_type
    
    from cvxopt import matrix,solvers

    K=np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            kernel= {'linear':np.dot(x[i], x[j]),'polynomial':(c + np.dot(x[i], x[j])) ** p,
                 'rbf':np.exp(-np.linalg.norm(x[i]-x[j])**2 /(2*(sigma ** 2)))}
            K[i,j]=kernel[kr_type]
    P=matrix(np.outer(y,y)*K)
    q=matrix(-1*np.ones((N)))
    A=matrix(np.transpose(y))
    b=matrix(np.zeros((1,1)))
    G=matrix(-1*np.identity(N))
    h=matrix(np.zeros((N,1)))
    sol = solvers.qp(P,q,G,h,A,b)
    al=np.ravel(sol['x'])    

    # Support vectors have non zero lagrange multipliers
    sv = al > 1e-5
    ind = np.arange(len(al))[sv]
    sv_al = al[sv]
    sv_x = x[sv]
    sv_y = y[sv]
    
    
    # Intercept
    b = 0
    for n in range(len(sv_al)):
        b += sv_y[n]
        b -= np.sum(sv_al*sv_y*K[ind[n],sv])
    b /= len(sv_al)
    
    # Weight vector
    if kr_type == 'linear':
        w = np.zeros(2)
        for n in range(len(sv_al)):
            w += sv_al[n] * sv_y[n] * sv_x[n]
    else:
        w = None
    
    
def predict(X,sigma=2,c=1,p=4):

    if w is not None:
        return np.sign(np.dot(X,w) + b)
    
    else:
        y_predict = np.zeros(len(X))
        for i in range(len(X)):
            sc = 0
            for n in range(len(sv_al)):
                kernel= {'polynomial':(c + np.dot(X[i], sv_x[n])) ** p,
                         'rbf':np.exp(-np.linalg.norm(X[i]-sv_x[n])**2 /(2*(sigma ** 2)))}
                sc += sv_al[n] * sv_y[n] *kernel[kr_type]
            y_predict[i] = sc
        return np.sign(y_predict + b)


def test(x_Q):
    pred=[]
    for i in range(len(x_Q)):
        c=predict(x_Q[i,:])
        pred.append(c)
    return np.array(pred)


    
  


fit(x,y,N,'linear')
pred_lin=test(x_Q)
print('linear prediction==',pred_lin)
fit(x,y,N,'polynomial')
pred_poly=test(x_Q)
print('polynomial prediction==',pred_poly)
fit(x,y,N,'rbf')
pred_rbf=test(x_Q)
print('rbf prediction==',pred_rbf)

     pcost       dcost       gap    pres   dres
 0: -1.1271e+02 -2.2136e+02  1e+03  3e+01  2e+00
 1: -2.5034e+02 -2.1702e+02  6e+02  2e+01  9e-01
 2: -3.8039e+02 -3.2597e+02  6e+02  2e+01  8e-01
 3: -1.7660e+03 -1.4691e+03  7e+02  1e+01  8e-01
 4: -1.1660e+03 -8.6137e+02  1e+03  1e+01  7e-01
 5: -9.2874e+02 -4.9246e+02  1e+03  8e+00  5e-01
 6: -1.9505e+02 -1.0495e+02  4e+02  2e+00  1e-01
 7: -5.9593e+01 -7.5655e+01  2e+01  2e-13  6e-14
 8: -6.9286e+01 -6.9501e+01  2e-01  1e-13  3e-14
 9: -6.9436e+01 -6.9438e+01  2e-03  4e-14  3e-14
10: -6.9437e+01 -6.9437e+01  2e-05  8e-14  3e-14
Optimal solution found.
linear prediction== [[[1.]]

 [[1.]]

 [[1.]]

 [[1.]]]
     pcost       dcost       gap    pres   dres
 0: -5.6549e+01 -1.0821e+02  1e+03  3e+01  2e+00
 1: -1.0401e+02 -8.2423e+01  5e+02  1e+01  8e-01
 2: -1.8501e+02 -1.3011e+02  5e+02  1e+01  7e-01
 3: -2.5113e+02 -1.4596e+02  5e+02  8e+00  5e-01
 4: -1.5062e+02 -4.9989e+01  3e+02  4e+00  2e-01
 5: -4.7065e+01 -1.8511e+01  9e+01  1e+0

The output predicted values 
Using linear kernel = [1,1,1,1]
Using polynomial kernel = [1,1,1,1]
Using rbf kernel = [1,1,1,1]